# Toeplitz Matrix
A matrix is Toeplitz if every diagonal from top-left to bottom-right has the same element.
For instance, the following matrix is Toeplitz:
$$
\begin{bmatrix}
1 & 2 & 3 & 4 \\
5 & 1 & 2 & 3 \\
9 & 5 & 1 & 2
\end{bmatrix}
$$
A square matrix is Toeplitz if and only if each of its $n-1$ diagonals is constant.
So a general 4x4 matrix looks like this:
$$
\begin{bmatrix}
a & b & c & d \\
e & a & b & c \\
f & e & a & b \\
g & f & e & a
\end{bmatrix}
$$

### Why we care about Toeplitz matrix?
Toeplitz Ststem $Tx = b$ is more efficient to solve than general system $Ax = b$.

# Block encoding
Block encoding is a technique that:
$$
\begin{align}
\forall A \in \mathbb{C}^{2^n \times 2^n}, \exists U \in \mathbb{C}^{2^{n+m} \times 2^{n+m}} \text{ s.t. }\\
(<0|^{\otimes 2^m} \otimes A I^{\otimes 2^n})U(|0>^{\otimes 2^m} \otimes I^{\otimes 2^n}) = \alpha A
\end{align}
$$
and 
$$
U=
\begin{bmatrix}
\frac{A}{\alpha} & * \\
* & *
\end{bmatrix}
$$


## 1. Constructing Toeplitz matrix based on banded, circulant matrix
The general Toeplitz matrix can be eaxpanded as
$$
\begin{bmatrix}
a & b & c & d & 0 & g & f & e \\
e & a & b & c & d & 0 & g & f \\
f & e & a & b & c & d & 0 & g \\
g & f & e & a & b & c & d & 0 \\
0 & g & f & e & a & b & c & d \\
d & 0 & g & f & e & a & b & c \\
c & d & 0 & g & f & e & a & b \\
b & c & d & 0 & g & f & e & a
\end{bmatrix}
$$
So we can use 7-banded matrix to block encode the Toeplitz matrix.
More generally, For a Toeplitz matrix of size $2^n \times 2^n$, we can use put it into a $2^{n+1} \times 2^{n+1}$ matrix with 2^{n+1}-1 banded matrix.
so the final cost of ancilla qubit is approximately $log( 2^{n+1}) + 1 = n+2$.

Let's take a = 1, b = 2, c = 3, d = 4, e = 5, f = 6, g = 7 as an example.

### 1.1 Constructing banded matrix
We need a banded matrix of shift 0, 1, 3, 4, -1, -2, -3.

In [101]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import XGate
from qiskit.quantum_info import Operator
import qiskit.visualization as qv
Add_1_circuit = QuantumCircuit(3)
Add_1_circuit.ccx(0, 1, 2)
Add_1_circuit.cx(0, 1)
Add_1_circuit.x(0)
qv.circuit_drawer(Add_1_circuit)
print(Add_1_circuit)
Add_1 = Add_1_circuit.to_gate(label='Add_1')

Minus_1_circuit = QuantumCircuit(3)
Minus_1_circuit.x(0)
Minus_1_circuit.x(1)
Minus_1_circuit.ccx(0, 1, 2)
Minus_1_circuit.x(1)
Minus_1_circuit.cx(0, 1)
qv.circuit_drawer(Minus_1_circuit)
print(Minus_1_circuit)
Minus_1 = Minus_1_circuit.to_gate(label='Minus_1')

B_circuit = QuantumCircuit(3)
B_circuit.append(Minus_1, [0, 1, 2])
B = B_circuit.to_gate(label='B')
C_circuit = QuantumCircuit(3)
C_circuit.append(Minus_1, [0, 1, 2])
C_circuit.append(Minus_1, [0, 1, 2])
C = C_circuit.to_gate(label='C')

D_circuit = QuantumCircuit(3)
D_circuit.append(Minus_1, [0, 1, 2])
D_circuit.append(Minus_1, [0, 1, 2])
D_circuit.append(Minus_1, [0, 1, 2])
D = D_circuit.to_gate(label='D')
E_circuit = QuantumCircuit(3)
E_circuit.append(Add_1, [0, 1, 2])
E = E_circuit.to_gate(label='E')
F_circuit = QuantumCircuit(3)
F_circuit.append(Add_1, [0, 1, 2])
F_circuit.append(Add_1, [0, 1, 2])
F = F_circuit.to_gate(label='F')
G_circuit = QuantumCircuit(3)
G_circuit.append(Add_1, [0, 1, 2])
G_circuit.append(Add_1, [0, 1, 2])
G_circuit.append(Add_1, [0, 1, 2])
G = G_circuit.to_gate(label='G')

               ┌───┐
q_0: ──■────■──┤ X ├
       │  ┌─┴─┐└───┘
q_1: ──■──┤ X ├─────
     ┌─┴─┐└───┘     
q_2: ┤ X ├──────────
     └───┘          
     ┌───┐               
q_0: ┤ X ├──■─────────■──
     ├───┤  │  ┌───┐┌─┴─┐
q_1: ┤ X ├──■──┤ X ├┤ X ├
     └───┘┌─┴─┐└───┘└───┘
q_2: ─────┤ X ├──────────
          └───┘          


Then we need to additional 3 qubits to store the value of a, b, c, d, e, f, g to prepare the addition of the banded matrix.

In [102]:
import numpy as np
c0 = np.sqrt(15 / 28)
c1 = np.sqrt(2 / 5)
c2 = np.sqrt(1 / 6)
c3 = (np.sqrt(14 / 13) + np.sqrt(12 / 13)) / 2
c4 = (np.sqrt(10 / 9) + np.sqrt(8 / 9)) / 2
c5 = (np.sqrt(6 / 5) + np.sqrt(4 / 5)) / 2

if c3 < 1 and c4 < 1 and c5 < 1:
    print("the value is valid")
else:
    print("the value is invalid")

theta_0 = 2 * np.arccos(c0)
theta_1 = 2 * np.arccos(c1)
theta_2 = 2 * np.arccos(c2)
theta_3 = 2 * np.arccos(c3)
theta_4 = 2 * np.arccos(c4)
theta_5 = 2 * np.arccos(c5)

the value is valid


In [103]:
R2 = QuantumCircuit(1)
R2.ry(theta_2, 0)
R2 = R2.to_gate(label='R2')
cR2 = R2.control(2)
R3 = QuantumCircuit(1)
R3.ry(theta_3, 0)
R3 = R3.to_gate(label='R3')
cR3 = R3.control(2)
R4 = QuantumCircuit(1)
R4.ry(theta_4, 0)
R4 = R4.to_gate(label='R4')
cR4 = R4.control(2)
R5 = QuantumCircuit(1)
R5.ry(theta_5, 0)
R5 = R5.to_gate(label='R5')
cR5 = R5.control(2)
hadamar = QuantumCircuit(1)
hadamar.h(0)
hadamar = hadamar.to_gate(label='H')
hadamard_controlled = hadamar.control(2)

In [104]:
import numpy as np
prep_ = QuantumRegister(3, 'prep_')
PREP_circuit = QuantumCircuit(prep_)

PREP_circuit.ry(theta_0, prep_[0])

PREP_circuit.ch(prep_[0], prep_[1])


PREP_circuit.x(prep_[2])
PREP_circuit.append(cR3, [prep_[0], prep_[2], prep_[1]])
PREP_circuit.x(prep_[2])

PREP_circuit.ccx(prep_[0], prep_[1], prep_[2])

PREP_circuit.x(prep_[0])
PREP_circuit.cry(theta_1, prep_[0], prep_[1])

PREP_circuit.append(hadamard_controlled, [prep_[0], prep_[1], prep_[2]])

PREP_circuit.x(prep_[1])
PREP_circuit.append(cR2, [prep_[0], prep_[1], prep_[2]])

PREP_circuit.x(prep_[0])

PREP_circuit.append(hadamard_controlled, [prep_[1], prep_[2], prep_[0]])
PREP_circuit.x(prep_[1])

PREP_circuit.x(prep_[0])
PREP_circuit.append(cR4, [prep_[0], prep_[1], prep_[2]])
PREP_circuit.x(prep_[0])

PREP_circuit.x(prep_[1])
PREP_circuit.append(cR5, [prep_[1], prep_[2], prep_[0]])
PREP_circuit.x(prep_[1])

qv.circuit_drawer(PREP_circuit)
print(PREP_circuit)

         ┌────────────┐                     ┌───┐                              »
prep__0: ┤ Ry(1.4993) ├──■────■──────────■──┤ X ├──────■─────────■─────────■───»
         └────────────┘┌─┴─┐┌─┴──┐       │  └───┘┌─────┴──────┐  │  ┌───┐  │   »
prep__1: ──────────────┤ H ├┤ R3 ├───────■───────┤ Ry(1.7722) ├──■──┤ X ├──■───»
             ┌───┐     └───┘└─┬──┘┌───┐┌─┴─┐     └────────────┘┌─┴─┐└───┘┌─┴──┐»
prep__2: ────┤ X ├────────────■───┤ X ├┤ X ├───────────────────┤ H ├─────┤ R2 ├»
             └───┘                └───┘└───┘                   └───┘     └────┘»
«         ┌───┐┌───┐┌───┐      ┌───┐┌────┐     
«prep__0: ┤ X ├┤ H ├┤ X ├──■───┤ X ├┤ R5 ├─────
«         └───┘└─┬─┘├───┤  │   ├───┤└─┬──┘┌───┐
«prep__1: ───────■──┤ X ├──■───┤ X ├──■───┤ X ├
«                │  └───┘┌─┴──┐└───┘  │   └───┘
«prep__2: ───────■───────┤ R4 ├───────■────────
«                        └────┘                


In [105]:
from qiskit.quantum_info import Operator
Prep_result = Operator(PREP_circuit).data
print(Prep_result)

[[ 1.88982237e-01+4.71844785e-16j -1.75932888e-01-4.44089210e-16j
  -2.31455025e-01-5.55111512e-16j  2.15472902e-01+4.44089210e-16j
  -4.22577127e-01-8.88178420e-16j  3.93397896e-01+9.15933995e-16j
   5.17549170e-01+1.22124533e-15j -4.81812056e-01-1.19348975e-15j]
 [ 4.62910050e-01+5.55111512e-16j  4.97245158e-01+6.38378239e-16j
   5.00000000e-01+6.38378239e-16j  5.37086156e-01+6.66133815e-16j
   1.73472348e-18+5.23852945e-32j -2.60208521e-17+6.47112461e-32j
   2.42861287e-17-6.16297582e-33j  0.00000000e+00-7.39557099e-32j]
 [ 3.77964473e-01-2.62542724e-17j -3.51865775e-01+2.47099034e-17j
   3.08606700e-01-2.16211655e-17j -2.87297202e-01+2.16211655e-17j
   4.22577127e-01+9.78622987e-16j -3.93397896e-01-8.92518139e-16j
   3.45032780e-01+7.88361909e-16j -3.21208037e-01-7.24941549e-16j]
 [ 0.00000000e+00+3.08148791e-33j  2.60208521e-17+6.49038391e-32j
   2.77555756e-17+3.70741514e-32j  1.47451495e-17-9.62964972e-35j
   4.81812056e-01+1.77635684e-15j  5.17549170e-01+1.88737914e-15j
  -4.81

In [106]:
prep_result = Prep_result @ np.array([1, 0, 0, 0, 0, 0, 0, 0]).T
print(prep_result.real ** 2 * 28)

[1. 6. 4. 0. 2. 3. 5. 7.]


In [107]:
cB = B.control(3)
cC = C.control(3)
cD = D.control(3)
cE = E.control(3)
cF = F.control(3)
cG = G.control(3)

a = 1, b = 2, c = 3, d = 4, e = 5, f = 6, g = 7
f, d, b, c, e, g

In [108]:
j_ = QuantumRegister(3, 'j_')
Select_circuit = QuantumCircuit(j_, prep_)

Select_circuit.x(prep_[2])
Select_circuit.x(prep_[1])
Select_circuit.append(cF,prep_[:] + j_[:])
Select_circuit.x(prep_[1])

Select_circuit.x(prep_[0])
Select_circuit.append(cD,prep_[:] + j_[:])
Select_circuit.x(prep_[2])
Select_circuit.x(prep_[1])

Select_circuit.append(cB,prep_[:] + j_[:])
Select_circuit.x(prep_[0])

Select_circuit.append(cC,prep_[:] + j_[:])
Select_circuit.x(prep_[1])
Select_circuit.x(prep_[0])

Select_circuit.append(cE,prep_[:] + j_[:])
Select_circuit.x(prep_[0])

Select_circuit.append(cG,prep_[:] + j_[:])

qv.circuit_drawer(Select_circuit)
print(Select_circuit)

              ┌────┐     ┌────┐     ┌────┐     ┌────┐     ┌────┐     ┌────┐
   j__0: ─────┤0   ├─────┤0   ├─────┤0   ├─────┤0   ├─────┤0   ├─────┤0   ├
              │    │     │    │     │    │     │    │     │    │     │    │
   j__1: ─────┤1 F ├─────┤1 D ├─────┤1 B ├─────┤1 C ├─────┤1 E ├─────┤1 G ├
              │    │     │    │     │    │     │    │     │    │     │    │
   j__2: ─────┤2   ├─────┤2   ├─────┤2   ├─────┤2   ├─────┤2   ├─────┤2   ├
              └─┬──┘┌───┐└─┬──┘     └─┬──┘┌───┐└─┬──┘┌───┐└─┬──┘┌───┐└─┬──┘
prep__0: ───────■───┤ X ├──■──────────■───┤ X ├──■───┤ X ├──■───┤ X ├──■───
         ┌───┐  │   ├───┤  │   ┌───┐  │   └───┘  │   ├───┤  │   └───┘  │   
prep__1: ┤ X ├──■───┤ X ├──■───┤ X ├──■──────────■───┤ X ├──■──────────■───
         ├───┤  │   └───┘  │   ├───┤  │          │   └───┘  │          │   
prep__2: ┤ X ├──■──────────■───┤ X ├──■──────────■──────────■──────────■───
         └───┘                 └───┘                                       


In [109]:
Select = Select_circuit.to_gate(label='Select')
PREP = PREP_circuit.to_gate(label='PREP')
PREP_dag_circuit = PREP_circuit.inverse()
PREP_dag = PREP_dag_circuit.to_gate(label='PREP_dag')

In [110]:
Final_circuit = QuantumCircuit(j_, prep_)
Final_circuit.append(PREP, prep_[:])
Final_circuit.append(Select, j_[:] + prep_[:])
Final_circuit.append(PREP_dag, prep_[:])

qv.circuit_drawer(Final_circuit)
print(Final_circuit)

                  ┌─────────┐             
   j__0: ─────────┤0        ├─────────────
                  │         │             
   j__1: ─────────┤1        ├─────────────
                  │         │             
   j__2: ─────────┤2        ├─────────────
         ┌───────┐│  Select │┌───────────┐
prep__0: ┤0      ├┤3        ├┤0          ├
         │       ││         ││           │
prep__1: ┤1 PREP ├┤4        ├┤1 PREP_dag ├
         │       ││         ││           │
prep__2: ┤2      ├┤5        ├┤2          ├
         └───────┘└─────────┘└───────────┘


In [111]:
Fonal_result = Operator(Final_circuit).data
print(Fonal_result)

[[ 3.57142857e-02+2.62532034e-15j  7.14285714e-02+8.27046320e-15j
   1.07142857e-01+7.65729096e-15j ...  5.94763095e-16+4.43886571e-15j
  -7.14056735e-16-1.37807450e-15j  1.21405227e-01-5.79719640e-15j]
 [ 1.78571429e-01+3.32066823e-15j  3.57142857e-02+2.99312074e-15j
   7.14285714e-02+1.03546375e-14j ... -8.87268264e-16-3.93817484e-15j
   9.85942529e-16+3.51278246e-15j -5.60926901e-16-1.58009586e-15j]
 [ 2.14285714e-01+9.22002432e-16j  1.78571429e-01+2.68826859e-15j
   3.57142857e-02+6.29210567e-16j ... -1.21405227e-01+6.57253820e-15j
  -1.30127519e-15-4.38445291e-15j  1.41748980e-15+4.94560566e-15j]
 ...
 [-1.21405227e-01+8.92756242e-15j  8.37665667e-16-4.47343192e-15j
  -1.52986254e-15+3.78869973e-15j ...  5.00000000e-01-6.25658337e-15j
   2.88550418e-01+2.22211430e-15j  2.57352961e-02+5.67909581e-15j]
 [-5.25105052e-02+1.25895256e-15j -1.21405227e-01+6.64620205e-15j
   9.50551537e-16-4.91435562e-15j ...  8.25396825e-02+4.16985901e-15j
   5.00000000e-01-3.68457934e-15j  2.88550418e-

In [112]:
result = Fonal_result[0:4, 0:4]
print(np.round(result.real * 28, 1))

[[1. 2. 3. 4.]
 [5. 1. 2. 3.]
 [6. 5. 1. 2.]
 [7. 6. 5. 1.]]


### 1.2 Conclusion
If we need to add up n matrices, we need to $\lceil log_2(n)\rceil$ qubits to store the coefficients of the matrices.
However, if the n coefficients are different, then we need n - 1 rotations to prepare them, this is corollary of fundamental theorem of arithmetic.
To make it easier to calculate the angle of the rotation, it is better to try to make the coefficients symmetric.

We can use R0, R1, R2 to map |0>|0>|0> to |0>|0>|0>, |0>|0>|1>, |0>|1>|0>, |1>|0>|0>, respectively. Then, we can use a controlled Hadamard gate to achieve pairs of states with certain characteristics that have the same coefficients.

This statement means that specific quantum gates (R0, R1, R2) are used to achieve specific basis state mappings, and subsequently, a controlled Hadamard gate is utilized to generate combinations of quantum states that have specific relationships (such as two qubits of them being the same) and identical coefficients. This process creates a symmetric set of coefficients that can be used to construct a Toeplitz matrix.

Then use R3, R4, R5 to map a pair of states that have specific relationships (two qubits of them being the same) to a pair of states that have specific relationships (one is original coefficient times cosine add sine, the other is original coefficient times cosine minus sine). 

By these way we can create odd number of different coefficients.

In this way, for instance, we can get coefficients for different states as follows:
for 000, we can get $cos(\theta_0)cos(\theta_1)cos(\theta_2)$
for 001, we can get $\frac{1}{\sqrt{2}}sin(\theta_0)(cos(\theta_3) - sin(\theta_3))$
for 010, we can get $\frac{1}{\sqrt{2}}cos(\theta_0)sin(\theta_1)(cos(\theta_4) - sin(\theta_4))$
for 100, we can get $\frac{1}{\sqrt{2}}sin(\theta_0)cos(\theta_1)sin(\theta_2)(cos(\theta_5) - sin(\theta_5))$
for 101, we can get $\frac{1}{\sqrt{2}}cos(\theta_0)sin(\theta_1)sin(\theta_2)(cos(\theta_5) + sin(\theta_5))$
for 110, we can get $\frac{1}{\sqrt{2}}sin(\theta_0)sin(\theta_1)(cos(\theta_4) + sin(\theta_4))$
for 111, we can get $\frac{1}{\sqrt{2}}sin(\theta_0)(cos(\theta_3) + sin(\theta_3))$

#### 1.2.1 The feasibility of the process
To prove the feasibility of the process, we need that all the rotation angles are solvable.
That is to prove the $cos(\theta_i)$ and $sin(\theta_i)$ are solvable and the value of them is less than 1.

Let's use the example of 7-banded Toeplitz matrix to prove the feasibility of the process:
Let the absolute value of coefficients of the Toeplitz matrix be $a_0, a_1, a_2, a_3, a_4, a_5, a_6$ in increasing order.
Let $a_5, a_6$ be assigned to 001 and 111, $a_3, a_4$ be assigned to 010 and 110, $a_1, a_2$ be assigned to 100 and 101, $a_0$ be assigned to 000.
Then we can solve the equation system to get the value of $\theta_0, \theta_1, \theta_2, \theta_3, \theta_4, \theta_5$.

Let $a = a_0 + a_1 + a_2 + a_3 + a_4 + a_5 + a_6 = 28$.
So $sin(\theta_0) = \sqrt{\frac{a_5 + a_6}{a}}$, $cos(\theta_0) = \sqrt{\frac{a - a_5 - a_6}{a}}$.
Then we can get the value of $\theta_0$.
We can get the value of $\theta_0$.
$sin(\theta_1) = \sqrt{\frac{a_3 + a_4}{a}\frac{a}{a - a_5 - a_6}} = \sqrt{\frac{a_3 + a_4}{a - a_5 - a_6}}$, $cos(\theta_1) = \sqrt{\frac{a - a_3 - a_4 - a_5 - a_6}{a - a_5 - a_6}}$.
Then we can get the value of $\theta_1$.
$sin(\theta_2) = \sqrt{\frac{a_1 + a_2}{a}\frac{a - a_5 - a_6}{a - a_3 - a_4 - a_5 - a_6}\frac{a}{a - a_5 - a_6}} = \sqrt{\frac{a_1 + a_2}{a - a_3 - a_4 - a_5 - a_6}}$, $cos(\theta_2) = \sqrt{\frac{a - a_1 - a_2 - a_3 - a_4 - a_5 - a_6}{a - a_3 - a_4 - a_5 - a_6}} = \sqrt{\frac{a_0}{a - a_3 - a_4 - a_5 - a_6}}$.
Then we can get the value of $\theta_2$.
Substitute the value of $\theta_0, \theta_1, \theta_2$ into the equation system, we can get the value of $\theta_3, \theta_4, \theta_5$ by solving the equation system in the form of:
$$
\left\{
\begin{align}
cos(\theta) - sin(\theta) &= \sqrt{2}\frac{\sqrt{a_i}}{\sqrt{a_{i} + a_{i+1}}\\
cos(\theta) + sin(\theta) &= \sqrt{2}\frac{\sqrt{a_{i+1}}}{\sqrt{a_{i} + a_{i+1}}\\
\end{align}
\right.
$$
These equations are solvable. Because according to the arithmetic-quadratic mean inequality, we have:$\sqrt{2a_i} + \sqrt{2a_{i+1}} \leq \sqrt{4(a_i + a_{i+1})}$.
Thus, the whole process is feasible.


Using this method, no matter how many coefficients need to be prepared, one can construct the circuit and calculate the angles following these steps.

## 3. Lower bound of the number of ancilla qubits
a Toeplitz matrix of n by n has 2n-1 diagonals, the 

# Block Topelitz matrix

A block Toeplitz matrix is a kind of block matrix, which contains blocks that are repeated down the diagonals of the matrix, as a Toeplitz matrix has elements repeated down the diagonal.